In [31]:
!pip install transformers --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
from transformers import pipeline
import gradio as gr
import random

emotion_classifier = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion")

Device set to use cuda:0


In [33]:
playlists = {
    "joy": [
        "Happy - Pharrell Williams",
        "Can't Stop the Feeling! - Justin Timberlake",
        "Walking on Sunshine - Katrina & The Waves",
        "Uptown Funk - Mark Ronson ft. Bruno Mars",
        "Best Day of My Life - American Authors",
        "Good as Hell - Lizzo",
        "Shake It Off - Taylor Swift",
        "On Top of the World - Imagine Dragons",
        "I'm Yours - Jason Mraz",
        "Send Me On My Way - Rusted Root"
    ],
    "sadness": [
        "Someone Like You - Adele",
        "Fix You - Coldplay",
        "Stay With Me - Sam Smith",
        "Skinny Love - Bon Iver",
        "Everybody Hurts - R.E.M.",
        "The Night We Met - Lord Huron",
        "Lost Cause - Beck",
        "Yesterday - The Beatles",
        "Hurt - Johnny Cash",
        "I Will Always Love You - Whitney Houston"
    ],
    "anger": [
        "Break Stuff - Limp Bizkit",
        "Killing In The Name - Rage Against The Machine",
        "You Oughta Know - Alanis Morissette",
        "Bulls on Parade - Rage Against The Machine",
        "Smells Like Teen Spirit - Nirvana",
        "In the End - Linkin Park",
        "Duality - Slipknot",
        "Given Up - Linkin Park",
        "Fighter - Christina Aguilera",
        "No One Knows - Queens of the Stone Age"
    ],
    "fear": [
        "Everybody’s Got To Learn Sometime - Beck",
        "Disturbia - Rihanna",
        "Creep - Radiohead",
        "Mad World - Tears for Fears / Gary Jules",
        "Boulevard of Broken Dreams - Green Day",
        "Shake Me Down - Cage The Elephant",
        "Breathin - Ariana Grande",
        "Chandelier - Sia",
        "Breathe Me - Sia",
        "How to Save a Life - The Fray"
    ],
    "surprise": [
        "Surprise Yourself - Jack Garratt",
        "What a Wonderful World - Louis Armstrong",
        "A Sky Full of Stars - Coldplay",
        "Feel It Still - Portugal. The Man",
        "Good Time - Owl City & Carly Rae Jepsen",
        "Wake Me Up - Avicii",
        "Shut Up and Dance - Walk The Moon",
        "Counting Stars - OneRepublic",
        "Electric Feel - MGMT",
        "Happy Now - Kygo & Sandro Cavazza"
    ],
    "love": [
        "All of Me - John Legend",
        "Perfect - Ed Sheeran",
        "Make You Feel My Love - Adele",
        "Thinking Out Loud - Ed Sheeran",
        "Just the Way You Are - Bruno Mars",
        "Love on Top - Beyoncé",
        "Your Song - Elton John",
        "Can't Help Falling in Love - Elvis Presley",
        "I Don’t Want to Miss a Thing - Aerosmith",
        "At Last - Etta James"
    ]
}

def recommend_songs(text):
    result = emotion_classifier(text)[0]
    emotion = result['label'].lower()
    songs = playlists.get(emotion, playlists["joy"])
    recommended = random.sample(songs, 3)
    return emotion, recommended

# Testing

In [34]:
input_text = "I feel happy and exicted today!"
emotion, songs = recommend_songs(input_text)

print(f"Detected emotion: {emotion.capitalize()}")
print("Recommended Songs:")
for song in songs:
    print(f"-{song}")

Detected emotion: Joy
Recommended Songs:
-I'm Yours - Jason Mraz
-Good as Hell - Lizzo
-Best Day of My Life - American Authors


In [35]:
intface = gr.Interface(
    fn=recommend_songs,
    inputs=gr.Textbox(label="How do you feel?"),
    outputs=[gr.Textbox(label="Detected Emotion"), gr.Textbox(label="Recommended Songs")],
    title="Emotion based Music Recommender"
)
intface.launch()

* Running on local URL:  http://127.0.0.1:7869
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://d5b751c9cc4f3309cf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [36]:
import kagglehub

path = kagglehub.dataset_download("abdullahorzan/moodify-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/moodify-dataset


In [37]:
!pip install spotipy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
client_id = user_secrets.get_secret("spotify_client_id")
client_secret = user_secrets.get_secret("client_secret")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [39]:
def get_song_info(uri):
    try:
        track = sp.track(uri)
        name = track['name']
        artist = track['artists'][0]['name']
        link = track['external_urls']['spotify']
        return f'<a href="{link}" target="_blank">{name} - {artist}</a>'
    except:
        return "Unknown track"

In [41]:
import pandas as pd

df = pd.read_csv('/kaggle/input/moodify-dataset/278k_labelled_uri.csv')
target_moods = {
    0: "sad",
    1: "happy",
    2: "energetic",
    3: "calm"
}

#print(df)
df["mood"] = df["labels"].map(target_moods)
df = df.dropna(subset=["uri"])
def recommend_songs(mood):
    filtered_songs = df[df["mood"] == mood]
    
    if filtered_songs.empty:
        return ["No Songs found for this mood."]
        
    recommendations = filtered_songs.sample(n=5, replace=False)
    result = [get_song_info(row['uri']) for _, row in recommendations.iterrows()]
    return "<br>".join(result)
moods = df["mood"].unique().tolist()

demo = gr.Interface(
    fn=recommend_songs,
    inputs=gr.Dropdown(moods, label="Select your mood"),
    outputs=gr.HTML(label="Recommended Songs")
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7871
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://eb6c47212982788dfd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
